In [6]:
import pandas as pd
import folium
import random
from shapely.geometry import Point
import geopandas as gpd
from geopy.distance import geodesic

In [7]:
# Generazione dati fittizi per UP (Uffici Postali)
n_up = 10
up_data = {
    "id": [f"UP_{i}" for i in range(n_up)],
    "lat": [random.uniform(45.0, 45.2) for _ in range(n_up)],
    "lon": [random.uniform(9.0, 9.2) for _ in range(n_up)],
    "produzione_totale": [random.randint(5000, 20000) for _ in range(n_up)],
    "bollettini": [random.randint(1000, 5000) for _ in range(n_up)],
    "pagoPA_MAV": [random.randint(500, 3000) for _ in range(n_up)],
    "ricariche_postepay": [random.randint(500, 2000) for _ in range(n_up)],
    "pacchi": [random.randint(300, 1500) for _ in range(n_up)],
}
df_up = pd.DataFrame(up_data)

In [15]:
df_up

,id,lat,lon,produzione_totale,bollettini,pagoPA_MAV,ricariche_postepay,pacchi
1,UP_1,45.004090,9.131227,7348,2117.000000,2925,1119,571
2,UP_2,45.047516,9.096861,14970,1916.000000,594,1604,1167
3,UP_3,45.045422,9.037004,5692,2706.000000,1331,1426,881
4,UP_4,45.133195,9.109110,15721,2148.166667,1770,847,1265
5,UP_5,45.197443,9.099580,13007,4883.166667,1864,894,1101
6,UP_6,45.029753,9.088834,6059,3774.000000,2740,1781,1100
7,UP_7,45.099661,9.146675,13136,4392.000000,904,1194,1427
8,UP_8,45.047337,9.191387,18367,2968.000000,984,926,769
9,UP_9,45.198323,9.031120,17848,5492.166667,1531,1140,1474


In [8]:
# Generazione dati fittizi per LIS (punti vendita autorizzati)
n_lis = 15
lis_data = {
    "id": [f"LIS_{i}" for i in range(n_lis)],
    "lat": [random.uniform(45.0, 45.2) for _ in range(n_lis)],
    "lon": [random.uniform(9.0, 9.2) for _ in range(n_lis)],
    "abilitazione_bollettini": [random.choice([True, False]) for _ in range(n_lis)],
    "abilitazione_pagoPA_MAV": [random.choice([True, False]) for _ in range(n_lis)],
    "abilitazione_ricariche_postepay": [random.choice([True, False]) for _ in range(n_lis)],
    "abilitazione_pacchi": [random.choice([True, False]) for _ in range(n_lis)],
}
df_lis = pd.DataFrame(lis_data)

In [17]:
df_lis

,id,lat,lon,abilitazione_bollettini,abilitazione_pagoPA_MAV,abilitazione_ricariche_postepay,abilitazione_pacchi
0,LIS_0,45.161381,9.116740,True,True,True,False
1,LIS_1,45.137879,9.019965,False,False,True,True
2,LIS_2,45.120830,9.060033,True,False,True,False
3,LIS_3,45.078096,9.119793,True,True,True,True
4,LIS_4,45.152058,9.027327,False,True,True,True
5,LIS_5,45.083519,9.054631,True,True,True,False
6,LIS_6,45.159394,9.029146,False,True,False,True
7,LIS_7,45.134174,9.034063,False,True,True,False
8,LIS_8,45.162697,9.071186,True,False,False,False
9,LIS_9,45.070289,9.024430,True,True,False,False


In [9]:
# Generazione dati fittizi per Banche
n_banche = 8
banche_data = {
    "id": [f"BANCA_{i}" for i in range(n_banche)],
    "lat": [random.uniform(45.0, 45.2) for _ in range(n_banche)],
    "lon": [random.uniform(9.0, 9.2) for _ in range(n_banche)],
    "gruppo_bancario": [random.choice(["Gruppo A", "Gruppo B", "Gruppo C"]) for _ in range(n_banche)],
    "peso_competitivo": [random.uniform(0.5, 1.5) for _ in range(n_banche)],
}
df_banche = pd.DataFrame(banche_data)

In [10]:
# Funzione per calcolare la distanza geografica
def calcola_distanza(coord1, coord2):
    return geodesic(coord1, coord2).meters

In [11]:
# Simulazione chiusura di un UP
up_da_chiudere = df_up.iloc[0]  # Selezioniamo il primo UP per la chiusura
df_up = df_up[df_up["id"] != up_da_chiudere["id"]]  # Rimuoviamo l'UP chiuso

In [12]:
# Redistribuzione della produzione
percentuale_up = 0.5  # 50% della produzione va agli UP vicini
percentuale_lis = 0.3  # 30% va agli LIS abilitati
percentuale_competitor = 0.1  # 10% passa ai competitor
percentuale_digitale = 0.1  # 10% passa al digitale

In [13]:
# Trova gli UP e LIS più vicini
up_limitrofi = df_up.copy()
up_limitrofi["distanza"] = up_limitrofi.apply(lambda row: calcola_distanza((up_da_chiudere["lat"], up_da_chiudere["lon"]), (row["lat"], row["lon"])), axis=1)
up_limitrofi = up_limitrofi.sort_values("distanza").head(3)  # Consideriamo i 3 UP più vicini

lis_limitrofi = df_lis.copy()
lis_limitrofi["distanza"] = lis_limitrofi.apply(lambda row: calcola_distanza((up_da_chiudere["lat"], up_da_chiudere["lon"]), (row["lat"], row["lon"])), axis=1)
lis_limitrofi = lis_limitrofi.sort_values("distanza").head(3)  # Consideriamo i 3 LIS più vicini

In [18]:
# Assicuriamoci che LIS abbia le stesse colonne di produzione di UP
for col in ["bollettini", "pagoPA_MAV", "ricariche_postepay", "pacchi"]:
    if col not in df_lis.columns:
        df_lis[col] = 0  # Creiamo le colonne mancanti con valore 0

In [19]:
# Distribuzione della produzione tra UP e LIS limitrofi
for col in ["bollettini", "pagoPA_MAV", "ricariche_postepay", "pacchi"]:
    produzione = up_da_chiudere[col]
    quota_up = produzione * percentuale_up / len(up_limitrofi)
    quota_lis = produzione * percentuale_lis / len(lis_limitrofi)
    
    df_up.loc[up_limitrofi.index, col] += quota_up
    df_lis.loc[lis_limitrofi.index, col] = df_lis.loc[lis_limitrofi.index, col].fillna(0) + quota_lis

C:\Users\User\AppData\Local\Temp\ipykernel_23268\3010420888.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[313.9 313.9 313.9]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_lis.loc[lis_limitrofi.index, col] = df_lis.loc[lis_limitrofi.index, col].fillna(0) + quota_lis
C:\Users\User\AppData\Local\Temp\ipykernel_23268\3010420888.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[102.6 102.6 102.6]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_lis.loc[lis_limitrofi.index, col] = df_lis.loc[lis_limitrofi.index, col].fillna(0) + quota_lis
C:\Users\User\AppData\Local\Temp\ipykernel_23268\3010420888.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 

In [20]:
# Creazione della mappa
mappa = folium.Map(location=[45.1, 9.1], zoom_start=12)

In [24]:
# Aggiunta UP
for _, row in df_up.iterrows():
    folium.Marker(
        location=[row['lat'], row['lon']],
        popup=f"{row['id']} - Produzione: {row['produzione_totale']}",
        icon=folium.Icon(color="blue", icon="envelope")
    ).add_to(mappa)

# Aggiunta LIS
for _, row in df_lis.iterrows():
    folium.Marker(
        location=[row['lat'], row['lon']],
        popup=f"{row['id']}",
        icon=folium.Icon(color="green", icon="shopping-cart")
    ).add_to(mappa)

# Aggiunta Banche
for _, row in df_banche.iterrows():
    folium.Marker(
        location=[row['lat'], row['lon']],
        popup=f"{row['id']} - {row['gruppo_bancario']}",
        icon=folium.Icon(color="red", icon="usd")
    ).add_to(mappa)

In [25]:
# Salvataggio mappa
mappa.save("mappa_simulazione_scenario.html")
mappa